In [1]:
# copilot version

from typing import List, Optional, TypedDict

from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool
from langgraph.graph import END, StateGraph
from tavily import TavilyClient

# Define the state
# state = {
#     "topic": None,
#     "search_results": None,
#     "summary": None,
#     "quiz_question": None,
#     "patient_answer": None,
#     "evaluation": None,
# }

# Define the state schema using TypedDict
class state(TypedDict):
    topic: Optional[str]
    search_results: Optional[List[dict]]
    summary: Optional[str]
    quiz_question: Optional[str]
    patient_answer: Optional[str]
    evaluation: Optional[str]

# Tools
search_tool = TavilySearchResults()


C:\Users\brian.westerman\AppData\Local\Temp\ipykernel_33952\3228426919.py:30: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults()


In [ ]:

# Nodes
def ask_topic(state):
    print("What health topic or medical condition would you like to learn about?")
    topic = input("Topic: ")
    state["topic"] = topic
    return state

def search_medical_info(state):
    results = search_tool.run(state["topic"])
    state["search_results"] = results
    return state

def summarize_info(state):
    # Dummy summarization for illustration
    summary = f"Summary of {state['topic']}: {state['search_results'][0]['content'][:300]}..."
    state["summary"] = summary
    return state

def present_info(state):
    print("\nHere is the information:\n")
    print(state["summary"])
    input("\nPress Enter when you're ready for a quiz.")
    return state

def generate_quiz(state):
    # Dummy quiz generation
    question = f"What is one key point about {state['topic']}?"
    state["quiz_question"] = question
    return state

def present_quiz(state):
    print("\nQuiz Question:")
    print(state["quiz_question"])
    answer = input("Your answer: ")
    state["patient_answer"] = answer
    return state

def evaluate_answer(state):
    # Dummy evaluation logic
    correct = "Yes" if "key" in state["patient_answer"].lower() else "No"
    explanation = f"Your answer is {correct}. Refer to: {state['summary'][:150]}..."
    state["evaluation"] = explanation
    return state

def present_results(state):
    print("\nEvaluation:")
    print(state["evaluation"])
    return state

def ask_continue(state):
    choice = input("\nWould you like to learn about another topic? (yes/no): ").strip().lower()
    if choice == "yes":
        # Reset state
        for key in state:
            state[key] = None
        return state
    else:
        return END


In [4]:
# Build the graph
graph = StateGraph(state)
graph.add_node("ask_topic", RunnableLambda(ask_topic))
graph.add_node("search_info", RunnableLambda(search_medical_info))
graph.add_node("summarize", RunnableLambda(summarize_info))
graph.add_node("present_info", RunnableLambda(present_info))
graph.add_node("generate_quiz", RunnableLambda(generate_quiz))
graph.add_node("present_quiz", RunnableLambda(present_quiz))
graph.add_node("evaluate", RunnableLambda(evaluate_answer))
graph.add_node("present_results", RunnableLambda(present_results))
graph.add_node("ask_continue", RunnableLambda(ask_continue))

# Define edges
graph.set_entry_point("ask_topic")
graph.add_edge("ask_topic", "search_info")
graph.add_edge("search_info", "summarize")
graph.add_edge("summarize", "present_info")
graph.add_edge("present_info", "generate_quiz")
graph.add_edge("generate_quiz", "present_quiz")
graph.add_edge("present_quiz", "evaluate")
graph.add_edge("evaluate", "present_results")
graph.add_edge("present_results", "ask_continue")
graph.add_edge("ask_continue", "ask_topic")  # Loop if user wants to continue

# Compile and run
app = graph.compile()
app.invoke(state)

InvalidUpdateError: Expected dict, got <class '__main__.state'>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE